In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

pd.set_option('display.max_columns', None)
plt.rcParams["figure.figsize"] = (8, 6)
fontsize = 18
figsize = (8, 6)

In [3]:
detection_results = pd.read_csv("detection_results.csv", index_col=0)
detection_results

,AUROC,dataset_name,data_type,flow_name,embedding_name,hidden_dim
92,0.996072,10_cover,features,maf,simple,32
61,0.894957,10_cover,features,real_nvp,simple,32
96,0.977241,10_cover,features,planar,simple,32
45,0.927776,10_cover,latents,glow,none,32
42,0.933159,10_cover,latents,real_nvp,none,32
...,...,...,...,...,...,...
39,0.532056,47_yeast,latents,planar,none,32
8,0.554642,47_yeast,features,maf,none,384
1,0.417629,47_yeast,features,planar,none,384
29,0.476216,47_yeast,latents,glow,none,32


In [4]:
flat_auroc = detection_results.groupby(["dataset_name", "flow_name", "data_type"]).max("AUROC")["AUROC"]
flat_auroc *= 100
flat_auroc = flat_auroc.reset_index().sort_values("dataset_name")
flat_auroc["model_name"] = flat_auroc["flow_name"] + " " + flat_auroc["data_type"]
flat_auroc = pd.pivot_table(flat_auroc, values="AUROC", index="dataset_name", columns="model_name")
flat_auroc

model_name,glow features,glow latents,maf features,maf latents,planar features,planar latents,real_nvp features,real_nvp latents
dataset_name,,,,,,,,
02_annthyroid,86.835734,66.399028,66.020633,80.783696,80.660931,86.522781,90.680121,86.775335
10_cover,86.168740,92.777569,99.607185,86.200831,97.724074,86.900812,89.495728,93.315920
11_donors,85.606055,81.198543,65.608573,50.627743,86.504116,88.600047,97.500072,84.287648
16_http,99.987013,99.937767,99.353162,99.945736,99.353784,99.945821,99.998851,99.945949
22_magic.gamma,67.051344,68.709796,82.146560,72.616039,71.327145,73.847622,73.974561,73.016789
23_mammography,62.131494,76.725329,90.096163,82.600406,89.437672,83.954183,83.204794,83.471697
27_PageBlocks,93.554669,93.237141,93.847121,85.315701,93.164199,93.074291,92.673662,93.981561
28_pendigits,79.450039,73.764684,97.764430,84.028685,92.936115,83.904662,87.978287,86.113498
32_shuttle,99.838109,98.963729,99.365592,99.487356,99.254332,99.612291,99.851837,99.677067


In [5]:
flat_auroc.round(2).to_csv("flat_auroc.csv")

In [6]:
adbench_auroc = pd.read_csv("adbench_auroc.csv", index_col=0)
adbench_auroc

,COPOD,ECOD,PCA,CBLOF,IForest
dataset_name,,,,,
annthyroid,76.80,78.03,66.25,62.26,82.01
cover,88.64,93.42,93.73,89.30,86.74
donors,81.76,74.45,83.15,60.44,77.68
http,99.29,98.10,99.72,99.60,99.96
magic.gamma,68.33,64.36,67.22,75.13,73.25
mammography,90.69,90.75,88.72,83.74,86.39
PageBlocks,88.05,90.92,90.64,85.04,89.57
pendigits,90.68,91.22,93.73,90.40,94.76
shuttle,99.35,99.40,98.62,83.48,99.56


In [7]:
flat_auroc.index = adbench_auroc.index
all_auroc = pd.merge(adbench_auroc, flat_auroc, left_index=True, right_index=True)
all_auroc.round(2).to_csv("all_auroc.csv")
all_auroc.round(2)

,COPOD,ECOD,PCA,CBLOF,IForest,glow features,glow latents,maf features,maf latents,planar features,planar latents,real_nvp features,real_nvp latents
dataset_name,,,,,,,,,,,,,
annthyroid,76.80,78.03,66.25,62.26,82.01,86.84,66.40,66.02,80.78,80.66,86.52,90.68,86.78
cover,88.64,93.42,93.73,89.30,86.74,86.17,92.78,99.61,86.20,97.72,86.90,89.50,93.32
donors,81.76,74.45,83.15,60.44,77.68,85.61,81.20,65.61,50.63,86.50,88.60,97.50,84.29
http,99.29,98.10,99.72,99.60,99.96,99.99,99.94,99.35,99.95,99.35,99.95,100.00,99.95
magic.gamma,68.33,64.36,67.22,75.13,73.25,67.05,68.71,82.15,72.62,71.33,73.85,73.97,73.02
mammography,90.69,90.75,88.72,83.74,86.39,62.13,76.73,90.10,82.60,89.44,83.95,83.20,83.47
PageBlocks,88.05,90.92,90.64,85.04,89.57,93.55,93.24,93.85,85.32,93.16,93.07,92.67,93.98
pendigits,90.68,91.22,93.73,90.40,94.76,79.45,73.76,97.76,84.03,92.94,83.90,87.98,86.11
shuttle,99.35,99.40,98.62,83.48,99.56,99.84,98.96,99.37,99.49,99.25,99.61,99.85,99.68


In [8]:
a = np.copy(all_auroc.values)
a.sort(axis=1)
a[:, -3:].round(2)

array([[ 86.78,  86.84,  90.68],
       [ 93.73,  97.72,  99.61],
       [ 86.5 ,  88.6 ,  97.5 ],
       [ 99.96,  99.99, 100.  ],
       [ 73.97,  75.13,  82.15],
       [ 90.1 ,  90.69,  90.75],
       [ 93.55,  93.85,  93.98],
       [ 93.73,  94.76,  97.76],
       [ 99.68,  99.84,  99.85],
       [ 79.34,  79.35,  92.03],
       [ 90.41,  93.36,  93.85],
       [ 98.27,  98.3 ,  98.69],
       [ 84.87,  89.92,  98.5 ],
       [ 57.59,  62.62,  69.01],
       [ 51.7 ,  53.21,  55.46]])

In [9]:
print(all_auroc.style.format('{:.2f}').highlight_max(axis=1).to_latex())

\begin{tabular}{lrrrrrrrrrrrrr}
 &  COPOD & ECOD  & PCA   & CBLOF & IForest & glow features & glow latents & maf features & maf latents & planar features & planar latents & real_nvp features & real_nvp latents \\
dataset_name &  &  &  &  &  &  &  &  &  &  &  &  &  \\
annthyroid    & 76.80 & 78.03 & 66.25 & 62.26 & 82.01 & 86.84 & 66.40 & 66.02 & 80.78 & 80.66 & 86.52 & \background-coloryellow 90.68 & 86.78 \\
cover         & 88.64 & 93.42 & 93.73 & 89.30 & 86.74 & 86.17 & 92.78 & \background-coloryellow 99.61 & 86.20 & 97.72 & 86.90 & 89.50 & 93.32 \\
donors        & 81.76 & 74.45 & 83.15 & 60.44 & 77.68 & 85.61 & 81.20 & 65.61 & 50.63 & 86.50 & 88.60 & \background-coloryellow 97.50 & 84.29 \\
http          & 99.29 & 98.10 & 99.72 & 99.60 & 99.96 & 99.99 & 99.94 & 99.35 & 99.95 & 99.35 & 99.95 & \background-coloryellow 100.00 & 99.95 \\
magic.gamma   & 68.33 & 64.36 & 67.22 & 75.13 & 73.25 & 67.05 & 68.71 & \background-coloryellow 82.15 & 72.62 & 71.33 & 73.85 & 73.97 & 73.02 \\
mammog

In [10]:
embeddings_results = pd.read_csv("embeddings_results.csv", index_col=0)

In [11]:
embedding_names = [
    'simple',
    'periodic',
    'none',
]

dataset_names = [
    # '38_thyroid',
    '16_http',
    # '22_magic.gamma',
    # '44_Wilt',
    # '27_PageBlocks',
    # '34_smtp',
    # '28_pendigits',
    '2_annthyroid',
    # '47_yeast',
    # '40_vowels',
    '11_donors',
    # '10_cover',
    # '23_mammography',
    '32_shuttle',
    '33_skin',
]


embeddings_results = embeddings_results[(embeddings_results['embedding_name'].isin(embedding_names)) & 
                                        (embeddings_results['dataset_name'].isin(dataset_names)) & 
                                        (embeddings_results["flow_name"] == "real_nvp")]

In [12]:
flat_embeddings_auroc = embeddings_results.groupby(["dataset_name", "data_type", "flow_name", "embedding_name"]).max("AUROC")["AUROC"]
flat_embeddings_auroc *= 100
flat_embeddings_auroc = flat_embeddings_auroc.reset_index().sort_values("dataset_name")
flat_embeddings_auroc["model_name"] = flat_embeddings_auroc["flow_name"] + "_" + flat_embeddings_auroc["data_type"] + "_" + flat_embeddings_auroc["embedding_name"]
flat_embeddings_auroc = pd.pivot_table(flat_embeddings_auroc, values="AUROC", index="dataset_name", columns="model_name")
flat_embeddings_auroc.round(2)

model_name,real_nvp_features_none,real_nvp_features_periodic,real_nvp_features_simple,real_nvp_latents_none
dataset_name,,,,
11_donors,97.50,87.92,90.20,84.29
16_http,99.36,70.57,100.00,99.95
2_annthyroid,84.13,78.84,90.68,86.78
32_shuttle,99.37,83.97,99.85,99.68
33_skin,66.53,78.94,92.03,79.35


In [13]:
flat_embeddings_auroc.style.format('{:.2f}').highlight_max(axis=1)

model_name,real_nvp_features_none,real_nvp_features_periodic,real_nvp_features_simple,real_nvp_latents_none
dataset_name,,,,
11_donors,97.50,87.92,90.20,84.29
16_http,99.36,70.57,100.00,99.95
2_annthyroid,84.13,78.84,90.68,86.78
32_shuttle,99.37,83.97,99.85,99.68
33_skin,66.53,78.94,92.03,79.35


In [14]:
print(flat_embeddings_auroc.style.format('{:.2f}').highlight_max(axis=1).to_latex())

\begin{tabular}{lrrrr}
model_name & real_nvp_features_none & real_nvp_features_periodic & real_nvp_features_simple & real_nvp_latents_none \\
dataset_name &  &  &  &  \\
11_donors & \background-coloryellow 97.50 & 87.92 & 90.20 & 84.29 \\
16_http & 99.36 & 70.57 & \background-coloryellow 100.00 & 99.95 \\
2_annthyroid & 84.13 & 78.84 & \background-coloryellow 90.68 & 86.78 \\
32_shuttle & 99.37 & 83.97 & \background-coloryellow 99.85 & 99.68 \\
33_skin & 66.53 & 78.94 & \background-coloryellow 92.03 & 79.35 \\
\end{tabular}

